In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd() / 'src'))

import pandas as pd
import numpy as np
from mintic_project.data_loader import procesar_siniestros
from mintic_project.feature_engineering import DatasetPredictor

print('✓ Librerías cargadas')

## 1. Cargar y procesar datos

In [ ]:
# Descargar datos (pequeño dataset para pruebas)
df1, df2 = procesar_siniestros(directorio_salida='data', limite_registros=5000)
print(f'Datasets descargados: {df1.shape} y {df2.shape}')

## 2. Feature Engineering para predicción

In [ ]:
# Preparar datos para predicción
predictor = DatasetPredictor()
df_prediccion = predictor.preparar_dataset_completo(df1, df2)

print(f'Dataset procesado: {df_prediccion.shape}')
print(f'\nColumnas generadas: {list(df_prediccion.columns[:20])}')

## 3. Generar contexto para RAG

In [ ]:
# Generar contexto para RAG (Retrieval-Augmented Generation)
contexto_rag = predictor.generar_contexto_rag(df_prediccion)

print('Contextos generados para RAG:')
for tema, contexto in contexto_rag.items():
    print(f'\n--- {tema.upper()} ---')
    print(contexto[:200] + '...' if len(contexto) > 200 else contexto)

## 4. Ejemplo: Predicción simple con Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Preparar datos para modelo
# Usar 'gravedad' como target (si existe)
if 'gravedad' in df_prediccion.columns:
    # Filtrar columnas numéricas y encoded
    feature_cols = [col for col in df_prediccion.columns 
                   if 'encoded' in col or 'normalized' in col or col in ['edad', 'hora', 'mes', 'distancia_centro']]
    
    # Preparar X y y
    X = df_prediccion[feature_cols].fillna(0)
    y = df_prediccion['gravedad'].fillna('DESCONOCIDA')
    
    print(f'Features: {len(feature_cols)}')
    print(f'Clases objetivo: {y.unique()}')
    
    # Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Entrenar modelo
    print('\nEntrenando modelo Random Forest...')
    modelo = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    modelo.fit(X_train, y_train)
    
    # Evaluar
    score = modelo.score(X_test, y_test)
    print(f'✓ Accuracy: {score:.3f}')
    
    # Importancia de features
    importancia = pd.DataFrame({
        'feature': feature_cols,
        'importancia': modelo.feature_importances_
    }).sort_values('importancia', ascending=False)
    
    print(f'\nTop 10 features más importantes:')
    print(importancia.head(10).to_string())
else:
    print('Columna "gravedad" no disponible')

## 5. Integración con LangChain (ejemplo básico)

In [ ]:
# Ejemplo de cómo usar el contexto con LangChain
# Nota: Requiere OPENAI_API_KEY configurada en .env

ejemplo_prompt = f"""
Contexto de datos de siniestros viales:
{contexto_rag.get('general', 'N/A')}

Distribución por jornada:
{contexto_rag.get('jornada', 'N/A')}

Pregunta: ¿En qué jornada ocurren más siniestros?
Basándote en los datos, proporciona un análisis.
"""

print('Ejemplo de prompt para LangChain:')
print(ejemplo_prompt)

# Para usar con OpenAI:
# from langchain.llms import OpenAI
# llm = OpenAI(temperature=0.7)
# respuesta = llm(ejemplo_prompt)
# print(respuesta)

## 6. Guardar modelo y datos procesados

In [ ]:
import pickle

# Guardar dataset procesado
df_prediccion.to_parquet('data/siniestros_procesados.parquet', index=False)
print('✓ Dataset procesado guardado: data/siniestros_procesados.parquet')

# Guardar modelo (si se entrenó)
if 'modelo' in locals():
    with open('models/modelo_prediccion.pkl', 'wb') as f:
        pickle.dump(modelo, f)
    print('✓ Modelo guardado: models/modelo_prediccion.pkl')

# Guardar contexto RAG
import json
with open('data/contexto_rag.json', 'w', encoding='utf-8') as f:
    json.dump(contexto_rag, f, indent=2, ensure_ascii=False)
print('✓ Contexto RAG guardado: data/contexto_rag.json')